# Segmenting and Clustering Neighborhoods in Toronto part 1

### Data scrapping by BeautifulSoup library

In [4]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

tbl = soup.find("table",{"PostalCode":""})

tor_df = pd.read_html(str(tbl))[0]
print(tor_df.shape)
tor_df.head()

(180, 3)


Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [5]:
tor_df.drop(pd.Index(np.where(tor_df['Borough'].isin(["Not assigned"]))[0]), inplace = True)

tor_df.reset_index(drop=True, inplace=True)

tor_df.head()

Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [6]:
tor_df.sort_values(by='Postal Code', inplace=True)

tor_df.reset_index(inplace=True, drop=True)

tor_df.head()

Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

In [7]:
tor_df.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto part 2

## CSV file with coordinates merged with Borough and Neighborhood data frame

In [8]:
tor_latlng=pd.read_csv("http://cocl.us/Geospatial_data")

tor_latlng.sort_values(by='Postal Code', inplace=True)

tor_latlng.reset_index(inplace=True, drop=True)

tor_latlng.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [9]:
tor_df['Latitude']=tor_latlng['Latitude']
tor_df['Longitude']=tor_latlng['Longitude']

tor_df.head()

Postal Code      Borough                            Neighborhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [13]:
tor_df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

## Segmenting and Clustering Part 3

### Download necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

ModuleNotFoundError: No module named 'folium'

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

### To get coordinates of Toronto we will use geopy library. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent to_explorer.

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating of Toronto map

In [11]:
# to create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# to add markers to map
for lat, lng, borough, neighborhood in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Borough'], tor_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## We choose to collect data for Downtown Toronto.
### Below we created new data frame for the Downtown Toronto.

In [14]:
downtown = tor_df[tor_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown.head()

Postal Code           Borough                 Neighborhood   Latitude  \
0         M4W  Downtown Toronto                     Rosedale  43.679563   
1         M4X  Downtown Toronto  St. James Town, Cabbagetown  43.667967   
2         M4Y  Downtown Toronto         Church and Wellesley  43.665860   
3         M5A  Downtown Toronto    Regent Park, Harbourfront  43.654260   
4         M5B  Downtown Toronto     Garden District, Ryerson  43.657162   

   Longitude  
0 -79.377529  
1 -79.367675  
2 -79.383160  
3 -79.360636  
4 -79.378937

### To find coordinates of Downtown Toronto we will use the same code as for Toronto

In [15]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


### Let's render map of Downtown Toronto

In [16]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown['Latitude'], downtown['Longitude'], downtown['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

## Define Foursquare credential and version

## Below is a hidden cell due to sensitive info

In [48]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


### To explore first neighborhood in Downtown Toronto:

In [18]:
downtown.loc[0,'Neighborhood']

'Rosedale'

#### Neighborhood's coordinates

In [19]:
nei_lat = downtown.loc[0, 'Latitude'] # neighborhood latitude value
nei_lng = downtown.loc[0, 'Longitude'] # neighborhood longitude value

nei_name = downtown.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(nei_name, 
                                                               nei_lat, 
                                                               nei_lng))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


### To explore top 70 venues within 300m radius
#### We will create GET request URL
##### The cell below is hidden due to sensitive info.

In [49]:
# The code was removed by Watson Studio for sharing.

### Let's sent GET request and examine the results

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee389657d583140723f5893'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 66,
  'suggestedBounds': {'ne': {'lat': 43.6590221027, 'lng': -79.37719117217293},
   'sw': {'lat': 43.653622097299994, 'lng': -79.38464102782706}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57eda381498ebe0e6ef40972',
       'name': 'UNIQLO ユニクロ',
       'location': {'address': '220 Yonge St',
        'crossStreet': 'at Dundas St W',
        'lat': 43.65591027779457,
        'lng': -79.38064099181345,
        'labeledLatLngs

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name      categories        lat        lng
0          UNIQLO ユニクロ  Clothing Store  43.655910 -79.380641
1   Ed Mirvish Theatre         Theater  43.655102 -79.379768
2            DAVIDsTEA        Tea Room  43.656973 -79.381040
3  Silver Snail Comics      Comic Shop  43.657031 -79.381403
4  Yonge-Dundas Square           Plaza  43.656054 -79.380495

In [24]:
# how many venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

66 venues were returned by Foursquare.


### To explore the rest of neighborhoods in Downtown Toronto, we will create a funtion:

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### To run the above function and to create new date frame:

In [26]:
downtown_venues = getNearbyVenues(names=downtown['Neighborhood'],
                                   latitudes=downtown['Latitude'],
                                   longitudes=downtown['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [27]:
print(downtown_venues.shape)
downtown_venues.head()

(688, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0                     Rosedale              43.679563              -79.377529   
1  St. James Town, Cabbagetown              43.667967              -79.367675   
2  St. James Town, Cabbagetown              43.667967              -79.367675   
3  St. James Town, Cabbagetown              43.667967              -79.367675   
4  St. James Town, Cabbagetown              43.667967              -79.367675   

                    Venue  Venue Latitude  Venue Longitude      Venue Category  
0    Mooredale House Pool       43.678674       -79.379311                Pool  
1             Cranberries       43.667843       -79.369407               Diner  
2                F'Amelia       43.667536       -79.368613  Italian Restaurant  
3  Butter Chicken Factory       43.667072       -79.369184   Indian Restaurant  
4              Murgatroid       43.667381       -79.369311          Restaurant

In [29]:
# how many venues returned for each neighborhood

downtown_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                             7   
CN Tower, King and Spadina, Railway Lands, Harb...                      5   
Central Bay Street                                                     32   
Christie                                                                7   
Church and Wellesley                                                   52   
Commerce Court, Victoria Hotel                                         70   
First Canadian Place, Underground city                                 70   
Garden District, Ryerson                                               60   
Harbourfront East, Union Station, Toronto Islands                      67   
Kensington Market, Chinatown, Grange Park                              57   
Queen's Park, Ontario Provincial Government                            13   
Regent Park, Harbourfront                                              15   
Richmond, Adelaide, King                                               59   
Rosedale                                                                1   
St. James Town                                                         19   
St. James Town, Cabbagetown                                            22   
Stn A PO Boxes                                                         40   
Toronto Dominion Centre, Design Exchange                               70   
University of Toronto, Harbord                                         22   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                              7   
CN Tower, King and Spadina, Railway Lands, Harb...                       5   
Central Bay Street                                                      32   
Christie                                                                 7   
Church and Wellesley                                                    52   
Commerce Court, Victoria Hotel                                          70   
First Canadian Place, Underground city                                  70   
Garden District, Ryerson                                                60   
Harbourfront East, Union Station, Toronto Islands                       67   
Kensington Market, Chinatown, Grange Park                               57   
Queen's Park, Ontario Provincial Government                             13   
Regent Park, Harbourfront                                               15   
Richmond, Adelaide, King                                                59   
Rosedale                                                                 1   
St. James Town                                                          19   
St. James Town, Cabbagetown                                             22   
Stn A PO Boxes                                                          40   
Toronto Dominion Centre, Design Exchange                                70   
University of Toronto, Harbord                                          22   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Berczy Park                                             7               7   
CN Tower, King and Spadina, Railway Lands, Harb...      5               5   
Central Bay Street                                     32              32   
Christie                                                7               7   
Church and Wellesley                                   52              52   
Commerce Court, Victoria Hotel                         70              70   
First Canadian Place, Underground city                 70              70   
Garden District, Ryerson                               60              60   
Harbourfront East, Union 

In [30]:
# how many unique categories

print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 167 uniques categories.


## To analyze each neighborhood

In [31]:
# one hot encoding
downtown_1hot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_1hot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_1hot.columns[-1]] + list(downtown_1hot.columns[:-1])
downtown_1hot = downtown_1hot[fixed_columns]

downtown_1hot.head()

Yoga Studio  Adult Boutique  Airport Food Court  Airport Gate  \
0            0               0                   0             0   
1            0               0                   0             0   
2            0               0                   0             0   
3            0               0                   0             0   
4            0               0                   0             0   

   Airport Lounge  Airport Terminal  American Restaurant  Arepa Restaurant  \
0               0                 0                    0                 0   
1               0                 0                    0                 0   
2               0                 0                    0                 0   
3               0                 0                    0                 0   
4               0                 0                    0                 0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  BBQ Joint  Bakery  \
0            0                    0                 0          0       0   
1            0                    0                 0          0       0   
2            0                    0                 0          0       0   
3            0                    0                 0          0       0   
4            0                    0                 0          0       0   

   Bank  Bar  Beer Bar  Beer Store  Belgian Restaurant  Bistro  Boat or Ferry  \
0     0    0         0           0                   0       0              0   
1     0    0         0           0                   0       0              0   
2     0    0         0           0                   0       0              0   
3     0    0         0           0                   0       0              0   
4     0    0         0           0                   0       0              0   

   Bookstore  Breakfast Spot  Bubble Tea Shop  Building  Burger Joint  \
0          0               0                0         0             0   
1          0               0                0         0             0   
2          0               0                0         0             0   
3          0               0                0         0             0   
4          0               0                0         0             0   

   Burrito Place  Bus Stop  Butcher  Café  Camera Store  Candy Store  \
0              0         0        0     0             0            0   
1              0         0        0     0             0            0   
2              0         0        0     0             0            0   
3              0         0        0     0             0            0   
4              0         0        0     0             0            0   

   Caribbean Restaurant  Cheese Shop  Chinese Restaurant  Church  \
0                     0            0                   0       0   
1                     0            0                   0       0   
2                     0            0                   0       0   
3                     0            0                   0       0   
4                     0            0                   0       0   

   Clothing Store  Cocktail Bar  Coffee Shop  College Gym  College Rec Center  \
0               0             0            0            0                   0   
1               0             0            0            0                   0   
2               0             0            0            0                   0   
3               0             0            0            0                   0   
4               0             0            0            0                   0   

   Colombian Restaurant  Comfort Food Restaurant  Comic Shop  Concert Hall  \
0                     0                        0           0             0   
1                     0                        0           0             0   
2                     0                        0           0             0   
3                     0                        0           0             0   
4                     0

In [32]:
downtown_1hot.shape

(688, 167)

In [33]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

downtown_grouped = downtown_1hot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
2                                  Central Bay Street     0.000000   
3                                            Christie     0.000000   
4                                Church and Wellesley     0.000000   
5                      Commerce Court, Victoria Hotel     0.000000   
6              First Canadian Place, Underground city     0.000000   
7                            Garden District, Ryerson     0.000000   
8   Harbourfront East, Union Station, Toronto Islands     0.000000   
9           Kensington Market, Chinatown, Grange Park     0.000000   
10        Queen's Park, Ontario Provincial Government     0.000000   
11                          Regent Park, Harbourfront     0.000000   
12                           Richmond, Adelaide, King     0.000000   
13                                           Rosedale     0.000000   
14                                     St. James Town     0.000000   
15                        St. James Town, Cabbagetown     0.000000   
16                                     Stn A PO Boxes     0.000000   
17           Toronto Dominion Centre, Design Exchange     0.000000   
18                     University of Toronto, Harbord     0.045455   

    Adult Boutique  Airport Food Court  Airport Gate  Airport Lounge  \
0         0.000000                 0.0           0.0             0.0   
1         0.000000                 0.2           0.2             0.2   
2         0.000000                 0.0           0.0             0.0   
3         0.000000                 0.0           0.0             0.0   
4         0.019231                 0.0           0.0             0.0   
5         0.000000                 0.0           0.0             0.0   
6         0.000000                 0.0           0.0             0.0   
7         0.000000                 0.0           0.0             0.0   
8         0.000000                 0.0           0.0             0.0   
9         0.000000                 0.0           0.0             0.0   
10        0.000000                 0.0           0.0             0.0   
11        0.000000                 0.0           0.0             0.0   
12        0.000000                 0.0           0.0             0.0   
13        0.000000                 0.0           0.0             0.0   
14        0.000000                 0.0           0.0             0.0   
15        0.000000                 0.0           0.0             0.0   
16        0.000000                 0.0           0.0             0.0   
17        0.000000                 0.0           0.0             0.0   
18        0.000000                 0.0           0.0             0.0   

    Airport Terminal  American Restaurant  Arepa Restaurant  Art Gallery  \
0                0.0             0.000000          0.000000     0.000000   
1                0.2             0.000000          0.000000     0.000000   
2                0.0             0.000000          0.000000     0.000000   
3                0.0             0.142857          0.000000     0.000000   
4                0.0             0.000000          0.000000     0.000000   
5                0.0             0.028571          0.000000     0.014286   
6                0.0             0.042857          0.000000     0.014286   
7                0.0             0.000000          0.000000     0.016667   
8                0.0             0.000000          0.000000     0.014925   
9                0.0             0.000000          0.017544     0.000000   
10               0.0             0.000000          0.000000     0.000000   
11               0.0             0.000000          0.000000     0.000000   
12               0.0             0.033898          0.000000     0.000000   
13               0.0             0.000000          0.000000     0.000000   
14               0.0             0.000000          0.000000

In [34]:
downtown_grouped.shape

(19, 167)

In [35]:
# each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0    Concert Hall  0.14
1  Breakfast Spot  0.14
2   Grocery Store  0.14
3    Liquor Store  0.14
4      Restaurant  0.14


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop   0.2
1  Airport Food Court   0.2
2        Airport Gate   0.2
3      Airport Lounge   0.2
4    Airport Terminal   0.2


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.38
1   Italian Restaurant  0.06
2       Sandwich Place  0.06
3  Japanese Restaurant  0.06
4                 Café  0.06


----Christie----
                 venue  freq
0        Grocery Store  0.29
1                 Café  0.14
2  American Restaurant  0.14
3          Candy Store  0.14
4          Coffee Shop  0.14


----Church and Wellesley----
                 venue  freq
0              Gay Bar  0.08
1          Coffee Shop  0.06
2  Japanese Restaurant  0.04
3     Ra

In [36]:
# To put the above results into pandas df

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
# create new df and display top venues

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park         Grocery Store   
1  CN Tower, King and Spadina, Railway Lands, Har...    Airport Food Court   
2                                 Central Bay Street           Coffee Shop   
3                                           Christie         Grocery Store   
4                               Church and Wellesley               Gay Bar   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0              Beer Bar        Clothing Store            Restaurant   
1          Airport Gate        Airport Lounge      Airport Terminal   
2    Italian Restaurant                  Café        Sandwich Place   
3           Coffee Shop   American Restaurant                  Café   
4           Coffee Shop      Ramen Restaurant   Japanese Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0          Liquor Store        Breakfast Spot          Concert Hall   
1           Coffee Shop           Wings Joint   Distribution Center   
2   Japanese Restaurant         Deli / Bodega              Pharmacy   
3           Candy Store   Japanese Restaurant   Distribution Center   
4          Burger Joint          Dessert Shop     Martial Arts Dojo   

  8th Most Common Venue       9th Most Common Venue     10th Most Common Venue  
0                 Diner           Electronics Store        Dumpling Restaurant  
1  Fast Food Restaurant              Farmers Market       Ethiopian Restaurant  
2           Pizza Place  Modern European Restaurant  Middle Eastern Restaurant  
3  Ethiopian Restaurant           Electronics Store        Dumpling Restaurant  
4            Hobby Shop              Ice Cream Shop         Italian Restaurant

# Cluster Neighborhoods

In [45]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

### Let's create new df with a cluster and top 10 venues

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

Postal Code           Borough                 Neighborhood   Latitude  \
0         M4W  Downtown Toronto                     Rosedale  43.679563   
1         M4X  Downtown Toronto  St. James Town, Cabbagetown  43.667967   
2         M4Y  Downtown Toronto         Church and Wellesley  43.665860   
3         M5A  Downtown Toronto    Regent Park, Harbourfront  43.654260   
4         M5B  Downtown Toronto     Garden District, Ryerson  43.657162   

   Longitude  Cluster Labels   1st Most Common Venue  \
0 -79.377529               2                    Pool   
1 -79.367675               1                    Café   
2 -79.383160               1                 Gay Bar   
3 -79.360636               1  Furniture / Home Store   
4 -79.378937               1             Coffee Shop   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Wings Joint          Dessert Shop  Ethiopian Restaurant   
1                 Restaurant           Coffee Shop           Pizza Place   
2                Coffee Shop      Ramen Restaurant   Japanese Restaurant   
3       Gym / Fitness Center                Bakery                  Park   
4  Middle Eastern Restaurant                  Café                 Hotel   

  5th Most Common Venue 6th Most Common Venue     7th Most Common Venue  \
0     Electronics Store   Dumpling Restaurant                Donut Shop   
1     Indian Restaurant                Bakery            Sandwich Place   
2          Burger Joint          Dessert Shop         Martial Arts Dojo   
3           Coffee Shop   Distribution Center  Mediterranean Restaurant   
4        Clothing Store                   Bar                  Tea Room   

  8th Most Common Venue  9th Most Common Venue 10th Most Common Venue  
0               Dog Run    Distribution Center         Discount Store  
1         Jewelry Store  Sri Lankan Restaurant    Japanese Restaurant  
2            Hobby Shop         Ice Cream Shop     Italian Restaurant  
3        Sandwich Place               Bus Stop     Light Rail Station  
4                 Plaza              Bookstore                   Lake

### Visualization

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters